In [39]:
import pandas as pd
import re
import numpy as np
import pandas as pd
from pprint import pprint
import nltk.corpus
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

In [86]:
df = pd.read_csv('cleaned_data.csv')
df['comment_without_stopwords_stemmed'] = df['comment_without_stopwords_stemmed'].fillna('')
other_words = ['bro', 'lah', 'esp', 'lol', 'hahah']

In [95]:
data = df.comment_without_stopwords_stemmed.values.tolist()

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


def html_removal(sentences):
    for i in range(len(sentences)):    
        sentences[i] = re.sub(r'http\S+', '', sentences[i])
    return sentences
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def other_words_removal(text_array):
    for i in range(len(other_words)):
         for j in range(len(text_array)):
                if i in text_array[j]:
                    text_array.remove(i)
    return text_array

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out        

In [96]:
data = html_removal(data)
data_words = list(sent_to_words(data))
data_words = other_words_removal(data_words)
print(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

[['exterior', 'step', 'back', 'interior', 'looks', 'great'], ['looks', 'like', 'lower', 'profil'], ['looks', 'like', 'toyota', 'wish'], ['exterior', 'much', 'changes', 'interior', 'swee'], ['corporate', 'styling', 'cue', 'class', 'dont', 'know', 'why', 'shape', 'profile', 'reminds', 'vito', 'lol'], ['looks', 'like', 'lower', 'profile', 'drag', 'coefficient', 'cd', 'mini', 'mpv', 'impress'], ['exterior', 'much', 'changes', 'interior', 'swee', 'steering', 'wheel', 'cls', 'air', 'vents', 'sls', 'one', 'sweet', 'car'], ['nice', 'mercy', 'really', 'getting', 'quite', 'right'], ['nice', 'esp', 'comes', 'singapore', 'turbo', 'dct'], ['definitely', 'size', 'toyota', 'wish', 'looking', 'it', 'like', 'size', 'vw', 'touran', 'guess', 'positioning', 'class', 'vw', 'golf', 'size', 'segment', 'class', 'vw', 'touran', 'mini', 'mpv', 'segment', 'maybe', 'blk', 'compete', 'vw', 'tiguan'], ['interior', 'nice', 'seems', 'like', 'merc', 'interior', 'getting', 'nicer', 'since', 'facelift', 'class'], ['exte

In [93]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=200,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [94]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.079*"class" + 0.039*"glb" + 0.028*"suv" + 0.021*"low" + '
  '0.019*"mercedes_benz" + 0.019*"top" + 0.019*"diesel" + 0.018*"small" + '
  '0.016*"bmw" + 0.016*"version"'),
 (1,
  '0.037*"engine" + 0.026*"drive" + 0.020*"change" + 0.016*"road" + '
  '0.015*"seat" + 0.014*"system" + 0.013*"front" + 0.013*"amg" + 0.012*"rear" '
  '+ 0.010*"speed"'),
 (2,
  '0.062*"car" + 0.030*"get" + 0.021*"new" + 0.020*"go" + 0.020*"year" + '
  '0.017*"say" + 0.016*"good" + 0.014*"time" + 0.014*"think" + 0.013*"also"'),
 (3,
  '0.054*"coe" + 0.026*"sale" + 0.025*"point" + 0.022*"guy" + 0.022*"sell" + '
  '0.020*"post" + 0.019*"cnc" + 0.019*"lah" + 0.018*"high" + 0.016*"true"'),
 (4,
  '0.069*"gla" + 0.038*"option" + 0.038*"one" + 0.035*"charge" + '
  '0.026*"leather" + 0.025*"red" + 0.024*"black" + 0.018*"press" + 0.016*"ron" '
  '+ 0.016*"gt"'),
 (5,
  '0.052*"side" + 0.049*"renew" + 0.044*"pump" + 0.042*"row" + '
  '0.041*"insurance" + 0.034*"company" + 0.025*"form" + 0.024*"product" + '
  '0.

In [56]:
len(data_words)

6775

In [65]:
data_words[:1]

[['exterior', 'step', 'back', 'interior', 'looks', 'great']]